## IPython notebook for histogram equalization

### Import all modules and functions needed for this example, also define some functions we'll need.

In [ ]:
import numpy as np
from skimage.io import imread
from skimage import img_as_float
from skimage.exposure import rescale_intensity
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


def bin_histogram(image, num_bins, min_val=None, max_val=None):
    if min_val is None:
        min_val = image.min()
    if max_val is None:
        max_val = image.max()

    # For each pixel, compute the index of the bin the corresponding gray belongs to.
    histo_ind = (np.floor(
        (image.ravel() - min_val) * (num_bins - 1) / (max_val - min_val))).astype(int)

    # Straight forward implementation to compute the histogram from the indices with a for loop.
    # num_pix = image.size
    # histo = np.zeros(num_bins, dtype=int)
    # for i in range(num_pix):
    #     histo[histo_ind[i]] = histo[histo_ind[i]] + 1

    # Fast implementation avoiding the for loop using numpy
    histo = np.bincount(histo_ind)
    return histo


def equalize_histo(image, num_bins):
    m, n = image.shape
    min_val = image.min()
    max_val = image.max()
    num_pixels = m*n
    H_f = bin_histogram(image, num_bins)
    G_f = np.cumsum(H_f)

    histo_ind = (np.floor((image[:, :] - min_val) *
                          (num_bins - 1) / (max_val - min_val))).astype(int)
    image = (max_val - min_val) * G_f[histo_ind[:, :]] / num_pixels + min_val
    return image


def plot_histogram(H_f, show_cdf_yticks=True):
    plt.bar(range(H_f.size), H_f, color='C0')
    ax1 = plt.gca()
    if show_cdf_yticks:
        ax1.tick_params('y', colors='C0')
    ax1.xaxis.grid(color='gray')
    ax1.yaxis.grid(color='gray')
    ax2 = plt.twinx()
    ax2.plot(np.cumsum(H_f), color='C1')
    if show_cdf_yticks:
        ax2.tick_params('y', colors='C1')
    else:
        ax2.yaxis.set_major_locator(ticker.NullLocator())
    ax2.set_xlim([0, H_f.size-1])
    ax2.set_ylim([0, None])


### Load an input image, plot it and its histogram.

In [ ]:
# Read the input image.
image = img_as_float(imread('astronaut.png', as_gray=True))

# Plot the input image
plt.subplot(1, 2, 1)
plt.imshow(image, interpolation='nearest', cmap=plt.cm.get_cmap('gray'), vmin=0, vmax=1)
plt.title('Input image')
plt.axis('off')

# Plot the histogram of the input image
H_f = bin_histogram(image, 256, 0, 1)
plt.subplot(1, 2, 2)
plot_histogram(H_f)
plt.title('$H_f$ and $G_f$ of input')
plt.show()

### Equalize the histogram of the input image, plot the equalized image and its histogram.

In [ ]:
# Equalize the input image
equalized_image = equalize_histo(rescale_intensity(image), 255)

# Plot the equalized image
plt.subplot(1, 2, 1)
plt.imshow(equalized_image, interpolation='nearest', cmap=plt.cm.get_cmap('gray'), vmin=0, vmax=1)
plt.title('equalized image')
plt.axis('off')

# Plot the histogram of the equalized image
H_f = bin_histogram(equalized_image, 256, 0, 1)
plt.subplot(1, 2, 2)
plot_histogram(H_f)
plt.title('$H_f$ and $G_f$ of equalized image')
plt.show()
